---

# Importar Modulos

---

In [ ]:
# Database connection and operations
import psycopg2
from psycopg2 import OperationalError

# Data manipulation and analysis
import pandas as pd

# Date and time handling
from datetime import datetime

# File system path operations
from pathlib import Path

# SQLAlchemy ORM - database engine and operations
from sqlalchemy import (
    create_engine,
    DateTime,
    Enum,
    ForeignKey,
    Float,
    func,
    Integer,
    String,
    text,
)

# SQLAlchemy ORM - model definitions and mapping
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

# SQLAlchemy ORM - session management
from sqlalchemy.orm import Session


from pathlib import Path

* utils funciones

In [ ]:


def borrar_registros_tabla(db: Session, table_name: str):
    """
    Deletes all records from a specified table using SQLAlchemy session.
    
    Args:
        db: SQLAlchemy Session object
        table_name: Name of the table to clear (e.g., 'items', 'matches')
    
    Returns:
        str: Message indicating the result of the operation
    """
    try:
        delete_query = text(f"DELETE FROM {table_name}")
        db.execute(delete_query)
        db.commit()
        mensaje = f"✅ Todos los registros de la tabla '{table_name}' han sido eliminados."
        print(mensaje)
        return mensaje
    except Exception as e:
        db.rollback()
        mensaje = f"❌ Error al eliminar registros de '{table_name}': {e}"
        print(mensaje)
        return mensaje

---

# A - Crear base de datos 

---

In [ ]:
# ============================================================================
# SECTION 1: DATABASE CONFIGURATION
# ============================================================================


# Define database directory path relative to this file
DB_DIR = Path('C:\\Users\\anmmu\\OneDrive\\Proyectos\\999_Repositorios_Github\\MELI_app\\services\\db')
# Define database file path
DB_PATH = DB_DIR / "database.db"
# Build SQLite connection string
DATABASE_URL = f"sqlite:///{DB_PATH}"

In [ ]:

# ============================================================================
# SECTION 2: DECLARATIVE BASE
# ============================================================================

class Base(DeclarativeBase):
    """Declarative base for all SQLAlchemy models."""
    pass


# ============================================================================
# SECTION 3: DATA MODELS
# ============================================================================

class Item(Base):
    """Table for storing marketplace items."""
    __tablename__ = "items"

    # Auto-incremented primary key
    id: Mapped[str] = mapped_column(String, primary_key=True)
    
    # Foreign key: item identifier string
    id_item: Mapped[str] = mapped_column(String, primary_key=True)
    # Item title with unique constraint
    title: Mapped[str] = mapped_column(String, unique=True, nullable=False)
    # Record creation timestamp with timezone
    created_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), nullable=False
    )
    # Record last update timestamp with automatic update
    updated_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), onupdate=func.now(), nullable=False
    )


class Match(Base):
    """Table for storing similarity matches between items."""
    __tablename__ = "matches"

    # Auto-incremented primary key
    id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    # Foreign key to first item
    id_item_1: Mapped[str] = mapped_column(ForeignKey("items.id"), nullable=False)
    # Cached title of first item for performance
    title_item_1: Mapped[str] = mapped_column(String, nullable=False)
    # Foreign key to second item
    id_item_2: Mapped[str] = mapped_column(ForeignKey("items.id"), nullable=False)
    # Cached title of second item for performance
    title_item_2: Mapped[str] = mapped_column(String, nullable=False)
    # Similarity score between 0 and 1
    score: Mapped[float] = mapped_column(Float, nullable=False)
    # Match status: positive, in progress, or negative
    status: Mapped[Enum] = mapped_column(
        Enum("positivo", "en progreso", "negativo", name="match_status_enum"), 
        nullable=False
    )
    # Record creation timestamp
    created_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), nullable=False
    )
    # Record last update timestamp
    updated_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), onupdate=func.now(), nullable=False
    )


class MatchBackup(Base):
    """Table for storing backup copies of matches."""
    __tablename__ = "matches_backup"

    # Auto-incremented primary key
    id: Mapped[int] = mapped_column(Integer, primary_key=True, autoincrement=True)
    # First item identifier (no foreign key to allow orphaned backups)
    id_item_1: Mapped[str] = mapped_column(String, nullable=False)
    # Cached title of first item
    title_item_1: Mapped[str] = mapped_column(String, nullable=False)
    # Second item identifier
    id_item_2: Mapped[str] = mapped_column(String, nullable=False)
    # Cached title of second item
    title_item_2: Mapped[str] = mapped_column(String, nullable=False)
    # Similarity score
    score: Mapped[float] = mapped_column(Float, nullable=False)
    # Backup status
    status: Mapped[Enum] = mapped_column(
        Enum("positivo", "en progreso", "negativo", name="match_backup_status_enum"), 
        nullable=False
    )
    # When backup was created
    created_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), nullable=False
    )
    # When backup was last modified
    updated_at: Mapped[datetime] = mapped_column(
        DateTime(timezone=True), default=func.now(), onupdate=func.now(), nullable=False
    )
    # When backup was restored (nullable if not yet restored)
    restored_at: Mapped[datetime] = mapped_column(DateTime(timezone=True), nullable=True)



In [ ]:
# ============================================================================
# SECTION 4: DATABASE INITIALIZATION
# ============================================================================

def init_db():
    """
    Initialize database: creates directory if needed, checks if database exists,
    and creates all tables if database is new.
    """
    # Create database directory with parents if it doesn't exist
    DB_DIR.mkdir(parents=True, exist_ok=True)
    
    # Check if database file already exists
    db_exists = DB_PATH.exists()
    
    print(f"Database path: {DB_PATH}")
    print(f"Database exists: {db_exists}")
    
    # Create database engine
    try:
        engine = create_engine(DATABASE_URL)
        engine.connect().close()
        print("✓ Database connection successful.")
    except Exception as e:
        print(f"✗ Error connecting to database: {e}")
        exit(1)
    
    # Create tables only if database is new (doesn't exist)
    if not db_exists:
        print("Creating new database tables...")
        Base.metadata.create_all(engine)
        print("✓ Database tables created successfully.")
    else:
        print("✓ Using existing database.")
    
    print("✓ Database initialization completed.")
    return engine



In [ ]:

# ============================================================================
# SECTION 5: MAIN ENTRY POINT
# ============================================================================

if __name__ == "__main__":
    init_db()



---

# B - Tabla con información a cargar a la db

---

In [ ]:
ruta = 'C:\\Users\\anmmu\\OneDrive\\Proyectos\\999_Repositorios_Github\\MELI_app\\data\\data_matches - dataset.csv'
df_item = pd.read_csv(ruta, sep=',', encoding='utf-8')
df_item.info()


In [ ]:
df_items = pd.concat([df_item[['ITEM_A', 'TITLE_A']].rename(columns={'ITEM_A': 'ITEM', 'TITLE_A': 'TITLE'}), df_item[['ITEM_B', 'TITLE_B']].rename(columns={'ITEM_B': 'ITEM', 'TITLE_B': 'TITLE'})  ], ignore_index=True)
df_items.drop_duplicates(inplace=True)
df_items.info()

---

# C - Conexión a la db 

---

* de forma local sqllite (preparacion previo despliegue)

In [ ]:
# Create SQLAlchemy engine using the existing DATABASE_URL
engine = create_engine(DATABASE_URL)
print(f"Connected to database: {DB_PATH}")

In [56]:
# reiniciar tablas
borrar_registros_tabla(db=Session(engine), table_name="items")
borrar_registros_tabla(db=Session(engine), table_name="matches")
borrar_registros_tabla(db=Session(engine), table_name="matches_backup")

✅ Todos los registros de la tabla 'items' han sido eliminados.
✅ Todos los registros de la tabla 'matches' han sido eliminados.
✅ Todos los registros de la tabla 'matches_backup' han sido eliminados.


"✅ Todos los registros de la tabla 'matches_backup' han sido eliminados."

In [ ]:
# Insertar registros de items desde el dataframe df_items
with Session(engine) as db:

    # Obtener el último id de toda la tabla
    last_item = db.query(Item).order_by(Item.id.desc()).first()
    if last_item:
        new_id = int(last_item.id)
    else:
        new_id = 0
    
    for _, row in df_items.iterrows():
        # Verificar si ya existe el item
        existing_item = db.query(Item).filter(Item.id_item == str(row['ITEM'])).first()
        
        if existing_item:
            print(f"⚠️ Item ya existe: id_item={row['ITEM']}, title='{row['TITLE']}' - Omitido")
            continue
        
        new_id = str(int(new_id) + 1)

        # Generar timestamp actual
        current_timestamp = datetime.now()
        
        # Crear e insertar registro en tabla items
        new_item = Item(
            id=new_id,
            id_item=str(row['ITEM']),
            title=row['TITLE'],
            created_at=current_timestamp,
            updated_at=current_timestamp
        )
        db.add(new_item)
    
    db.commit()

print(f"\n✅ Proceso completado. Total de items en DataFrame: {len(df_items)}")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39

✅ Proceso completado. Total de items en DataFrame: 39


In [61]:
# Query to verify the items table after bulk ingestion
with Session(engine) as db:
    # Count total records
    count_query = text("SELECT COUNT(*) FROM items")
    total_records = db.execute(count_query).scalar()
    print(f"Total records in 'items' table: {total_records}")
    
    # Fetch and display first 10 records
    select_query = text("SELECT id, id_item, title FROM items LIMIT 50")
    records = db.execute(select_query).fetchall()
    print("\nFirst 10 records:")
    for record in records:
        print(f"  id={record[0]}, id_item={record[1]}, title={record[2]}")

Total records in 'items' table: 39

First 10 records:
  id=1, id_item=514341, title=Baraja de Evangelion y Gundam W
  id=2, id_item=687643, title=FAX TELEFONICA EQUIPOS - URGENTE
  id=3, id_item=535665, title=autografo de Chilavert
  id=4, id_item=2139803, title=Juego de tweeter.
  id=5, id_item=2139565, title=Chomba color rojo.
  id=6, id_item=468237168, title=Locales En Venta
  id=7, id_item=63486251, title=Colección Vinilos Los Grandes Músicos
  id=8, id_item=63486703, title=Zapato Y Botas De Mujer
  id=9, id_item=63537003, title=Lote De 12pares Medias Soquetes 1/4 Toalla Algodon,no Envios
  id=10, id_item=63480361, title=Lote X10 Excelentes Ponchos Toalla  Infantil Disney 60x120cm
  id=11, id_item=63477742, title=Ch Africa Similar Edition 100 Ml - Tester De Free Shop
  id=12, id_item=63482316, title=Entrada Bad Religion- Biohazard Obras 1993
  id=13, id_item=634887722, title=Laser 4 Bocas Usado Para Reparar
  id=14, id_item=63474586, title=Calefactor Placa Peabody Living & Arts Urg

* desplegada ya como srvicio postgress

In [62]:
# Establecer una nueva conexión
conn = psycopg2.connect(
    dbname="meli_app_db",
    user="user",
    password="password",
    host="localhost",
    port="5432"
)

# Crear un cursor para ejecutar la consulta
cursor = conn.cursor()


## C.1 tabla de *ITEMS*

---

### C.1.1 Crear la tabla 'items' si no existe

In [ ]:

cursor.execute("""
CREATE TABLE IF NOT EXISTS items (
    id BIGINT PRIMARY KEY,
    id_item BIGINT,
    title TEXT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
)
""")
conn.commit()

### C.1.2 insertar registros del .csv

In [ ]:
# Rollback any failed transaction
conn.rollback()

# obtener fecha y hora actual
now = datetime.now()
print("Fecha y hora actual:", now)

# Insertar registros en la tabla 'items'
for id_num, row in df_items.iterrows():
    cursor.execute(
        "INSERT INTO items (id, id_item, title, created_at, updated_at) VALUES (%s, %s, %s, %s, %s)",
        (id_num, row['ITEM'], row['TITLE'], now, now)
    )
    print(f"Registro insertado: id={id_num}, id_item={row['ITEM']}, title={row['TITLE']}")

# Confirmar los cambios en la base de datos
conn.commit()

In [ ]:
# Rollback any failed transaction
conn.rollback()

# Insert multiple dummy records into matches table
dummy_records = [
    (98, 9999999999, 'Producto A'),
    (99, 9999999998, 'diferente')
]

now = datetime.now()
print("Fecha y hora actual:", now)

for id_num, id_item, title in dummy_records:
    cursor.execute(
        "INSERT INTO items (id, id_item, title, created_at, updated_at) VALUES (%s, %s, %s, %s, %s)",
        (id_num, id_item, title, now, now)
    )
    print(f"Registro insertado: id={id_num}, id_item={id_item}, title={title}")

conn.commit()


### C.1.3 Revision tabla ITEMS

In [ ]:
try:
    
    # Consulta para obtener los nombres de las columnas de la tabla 'items'
    cursor.execute("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = 'items';
    """)
    
    # Obtener los resultados
    headers = cursor.fetchall()
    print("Encabezados de la tabla 'items':", [header[0] for header in headers])


except OperationalError as e:
    print(f"Error: No se pudo conectar a la base de datos.")
    print(f"Detalle: {e}")

In [64]:
try:
    # Consulta para obtener los primeros 50 registros de la tabla 'items'
    cursor.execute("""
        SELECT * 
        FROM items
        LIMIT 50;
    """)
    
    # Obtener los resultados
    records = cursor.fetchall()
    for record in records:
        print(record)

except OperationalError as e:
    print(f"Error: No se pudo conectar a la base de datos.")
    print(f"Detalle: {e}")


('1', 'Baraja de Evangelion y Gundam W', datetime.datetime(2026, 2, 18, 11, 28, 16, 302982, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 11, 28, 16, 302982, tzinfo=datetime.timezone.utc), '514341')
('2', 'FAX TELEFONICA EQUIPOS - URGENTE', datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), '687643')
('3', 'autografo de Chilavert', datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), '535665')
('4', 'Juego de tweeter.', datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), '2139803')
('5', 'Chomba color rojo.', datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 11, 28, 16, 304760, 

### C.2.1 Reinicio de tabla ITEMS

In [ ]:
# Rollback any failed transaction
conn.rollback()

# Borrar todo el contenido de la tabla ITEMS
cursor.execute("DELETE FROM items")
conn.commit()
print("Todos los registros de la tabla 'items' han sido eliminados.")

## C.2 tabla de *MATCHES*

---

### C.2.1 Crear la tabla 'matches' si no existe

In [ ]:
# Crear la tabla 'matches' si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
                id SERIAL PRIMARY KEY,
                id_item_1 BIGINT,
                title_item_1 TEXT,
                id_item_2 BIGINT,
                title_item_2 TEXT,
                score FLOAT,
                status VARCHAR,
                created_at TIMESTAMP,
                updated_at TIMESTAMP
)
""")
conn.commit()


### C.2.2 insertar registros de prueba

In [ ]:
# Rollback any failed transaction
conn.rollback()

# Insert multiple dummy records into matches table
dummy_records = [
    (0, 9999999999, 'Producto A', 9999999999, 'Producto A', 1.0, 'positivo'),
    (1, 9999999999, 'Producto A', 9999999998, 'diferente', 0.0, 'negativo'),
    (2, 9999999998, 'diferente', 9999999998, 'diferente', 1.0, 'positivo')
]

now = datetime.now()
print("Fecha y hora actual:", now)

for id_num, id_prod_1, name_prod_1, id_prod_2, name_prod_2, score_val, status_val in dummy_records:
    cursor.execute(
        "INSERT INTO matches (id, id_item_1, title_item_1, id_item_2, title_item_2, score, status, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (id_num, id_prod_1, name_prod_1, id_prod_2, name_prod_2, score_val, status_val, now, now)
    )
    print(f"Registro insertado: id={id_num}, id_item_1={id_prod_1}, title_item_1={name_prod_1}, id_item_2={id_prod_2}, title_item_2={name_prod_2}, score={score_val}, status={status_val}")

conn.commit()


### C.2.3 Revision tabla MATCHES

In [82]:
try:
    # Rollback any failed transaction
    conn.rollback()

    # Consulta para obtener los primeros 50 registros de la tabla 'items'
    cursor.execute("""
        SELECT * 
        FROM matches
    """)
    
    # Obtener los resultados
    records = cursor.fetchall()
    for record in records:
        print(record)

except OperationalError as e:
    print(f"Error: No se pudo conectar a la base de datos.")
    print(f"Detalle: {e}")


(742, '514341', 'Baraja de Evangelion y Gundam W', '687643', 'FAX TELEFONICA EQUIPOS - URGENTE', 0.0, 'negativo', datetime.datetime(2026, 2, 18, 14, 0, 11, 57189, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 14, 6, 9, 820286, tzinfo=datetime.timezone.utc))
(743, '514341', 'Baraja de Evangelion y Gundam W', '535665', 'autografo de Chilavert', 0.0, 'negativo', datetime.datetime(2026, 2, 18, 14, 0, 11, 66125, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 14, 6, 9, 826140, tzinfo=datetime.timezone.utc))
(744, '514341', 'Baraja de Evangelion y Gundam W', '2139803', 'Juego de tweeter.', 0.0, 'negativo', datetime.datetime(2026, 2, 18, 14, 0, 11, 71394, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 18, 14, 6, 9, 831673, tzinfo=datetime.timezone.utc))
(745, '514341', 'Baraja de Evangelion y Gundam W', '2139565', 'Chomba color rojo.', 0.0, 'negativo', datetime.datetime(2026, 2, 18, 14, 0, 11, 76107, tzinfo=datetime.timezone.utc), datetime.datetime(202

### C.2.4 Reinicio de tabla MATCHES

In [83]:
# Rollback any failed transaction
conn.rollback()

# Borrar todo el contenido de la tabla MATCHES
cursor.execute("DELETE FROM matches")
conn.commit()
print("Todos los registros de la tabla 'matches' han sido eliminados.")

Todos los registros de la tabla 'matches' han sido eliminados.


## Cerrar sesión

In [ ]:
    
# Cerrar el cursor y la conexión
cursor.close()
conn.close()